In [1]:
from quant_utils.db_conn import DB_CONN_JJTG_DATA

import pandas as pd

file_path = "C:/Users/Wilcoxon/Desktop/配置excel/组合配置信息/"
file_list = [
    'portfolio_benchmark.xlsx',
    'portfolio_info.xlsx',
    'portfolio_constraint.xlsx',
    'portfolio_products_weights.xlsx',
]

for file in file_list:
    df = pd.read_excel(file_path + file)
    if "TICKER_SYMBOL" in df.columns:
        df["TICKER_SYMBOL"] = (
            df["TICKER_SYMBOL"]
            .astype(int)
            .astype(str)
            .str.rjust(6, "0")
        )
        df["TICKER_SYMBOL"] = df["TICKER_SYMBOL"].str.rjust(6, "0")
        df.loc[df["TICKER_SYMBOL"] == "000000", "TICKER_SYMBOL"] = None

    if "ALTERNATIVE_TICKER_SYMBOL" in df.columns:
        df["ALTERNATIVE_TICKER_SYMBOL"].fillna(0, inplace=True)
        df["ALTERNATIVE_TICKER_SYMBOL"] = (
            df["ALTERNATIVE_TICKER_SYMBOL"]
            .astype(int)
            .astype(str)
            .str.rjust(6, "0")
        )
        df.loc[df["ALTERNATIVE_TICKER_SYMBOL"] == "000000", "ALTERNATIVE_TICKER_SYMBOL"] = None
    DB_CONN_JJTG_DATA.upsert(df, table=file.split('.')[0])

C:\Users\Wilcoxon\AppData\Local\Temp\ipykernel_21592\2857444691.py:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["ALTERNATIVE_TICKER_SYMBOL"].fillna(0, inplace=True)
